# Colab Demo

## Dependencies and Imports

In [ ]:
#@title Install dependencies

!pip install -q torchaudio omegaconf

import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

## List models

In [ ]:
# see latest avaiable models
available_languages = list(models.tts_models.keys())
print(f'Available languages {available_languages}')

for lang in available_languages:
    _models = list(models.tts_models.get(lang).keys())
    print(f'Available models for {lang}: {_models}')

## V4

In [ ]:
import torch

language = 'ru'
model_id = 'v4_ru'
device = torch.device('cpu')

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

### Speakers

In [ ]:
model.speakers

### Text

In [ ]:
sample_rate = 48000
speaker = 'xenia'
put_accent=True
put_yo=True
example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра к+едров.'

audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate,
                        put_accent=put_accent,
                        put_yo=put_yo)
print(example_text)
display(Audio(audio, rate=sample_rate))

### SSML

In [ ]:
ssml_sample = """
              <speak>
              <p>
                  Когда я просыпаюсь, <prosody rate="x-slow">я говорю довольно медленно</prosody>.
                  Пот+ом я начинаю говорить своим обычным голосом,
                  <prosody pitch="x-high"> а могу говорить тоном выше </prosody>,
                  или <prosody pitch="x-low">наоборот, ниже</prosody>.
                  Пот+ом, если повезет – <prosody rate="fast">я могу говорить и довольно быстро.</prosody>
                  А еще я умею делать паузы любой длины, например, две секунды <break time="2000ms"/>.
                  <p>
                    Также я умею делать паузы между параграфами.
                  </p>
                  <p>
                    <s>И также я умею делать паузы между предложениями</s>
                    <s>Вот например как сейчас</s>
                  </p>
              </p>
              </speak>
              """

sample_rate = 48000
speaker = 'xenia'              
audio = model.apply_tts(ssml_text=ssml_sample,
                        speaker=speaker,
                        sample_rate=sample_rate)
display(Audio(audio, rate=sample_rate))

### Random speaker

#### Generate new

In [ ]:
sample_rate = 48000
speaker = 'random'

example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра к+едров.'

audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate)
print(example_text)
display(Audio(audio, rate=sample_rate))

#### Save generated speaker

In [ ]:
voice_path = 'test_voice.pt'
model.save_random_voice(voice_path)

#### Generate with saved speaker

In [ ]:
sample_rate = 48000
speaker = 'random'
voice_path = 'test_voice.pt'

example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра к+едров.'

audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate,
                        voice_path=voice_path)
print(example_text)
display(Audio(audio, rate=sample_rate))

## V2

### One speaker

In [ ]:
import torch

language = 'ru'
speaker = 'kseniya_v2'
sample_rate = 16000
device = torch.device('cpu')
model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=speaker)
model.to(device)  # gpu or cpu

In [ ]:
audio = model.apply_tts(texts=[example_text],
                        sample_rate=sample_rate)

print(example_text)
display(Audio(audio[0], rate=sample_rate))

### Multi speaker

In [ ]:
import torch

language = 'multi'
speaker = 'multi_v2'
sample_rate = 16000
device = torch.device('cpu')
model, avail_speakers = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_tts',
                                       language=language,
                                       speaker=speaker)
model.to(device)  # gpu or cpu

In [ ]:
speaker = 'dilyara'
example_text = avail_speakers['baya']['example']

In [ ]:
audio = model.apply_tts(texts=[example_text],
                        speakers=[speaker],
                        sample_rate=sample_rate)

print(example_text)
display(Audio(audio[0], rate=sample_rate))

## V1

These examples are for older models and obsolete

In [ ]:
import torch

language = 'ru'
speaker = 'kseniya_16khz'
device = torch.device('cpu')
model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
model = model.to(device)  # gpu or cpu

In [ ]:
audio = apply_tts(texts=[example_text],
                  model=model,
                  sample_rate=sample_rate,
                  symbols=symbols,
                  device=device)

print(example_text)
display(Audio(audio[0], rate=sample_rate))

### Enhance synthesis with logmmse

In [ ]:
!pip install -q logmmse

You can try to enhance synthesized audio with logmmse algorithm, though it could demand parameters tuning for the particular speaker.

In [ ]:
import numpy as np
from logmmse import logmmse

enhanced = logmmse(np.array(audio[0]), sample_rate, output_file=None, initial_noise=1, window_size=160, noise_threshold=0.15)
display(Audio(enhanced, rate=sample_rate))

# Minimal Example to Run Locally 

## Dependencies and Imports

In [ ]:
#@title Install dependencies

!pip install -q torch==1.10

## V4

In [ ]:
import os
import torch

device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model.pt'

if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v4_ru.pt',
                                   local_file)  

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)

example_text = 'В недрах тундры выдры в г+етрах т+ырят в вёдра ядра кедров.'
sample_rate = 48000
speaker='baya'

audio_paths = model.save_wav(text=example_text,
                             speaker=speaker,
                             sample_rate=sample_rate)

## V2

### One speaker

In [ ]:
import os
import torch

device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model.pt'

if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v2_kseniya.pt',
                                   local_file)  

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)

example_batch = ['В недрах тундры выдры в г+етрах т+ырят в вёдра ядра кедров.',
                 'Котики - это жидкость!',
                 'М+ама М+илу м+ыла с м+ылом.']
sample_rate = 16000

audio_paths = model.save_wav(texts=example_batch,
                             sample_rate=sample_rate)
print(audio_paths)

### Multi speaker

In [ ]:
import os
import torch

device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model_multi.pt'
speakers = ['aidar', 'baya', 'kseniya', 'irina', 'ruslan', 'natasha',
                    'thorsten', 'tux', 'gilles', 'lj', 'dilyara']

if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/multi/v2_multi.pt',
                                   local_file)  

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)

example_batch = ['В н+едрах т+ундры в+ыдры в г+етрах т+ырят в в+ёдра +ядра к+едров.',
                 'Fischers Fritze fischt frische Fische',
                 'Can you can a canned can into an un-canned can?']
example_speakers = ['baya', 'baya', 'baya']
sample_rate = 16000

audio_paths = model.save_wav(texts=example_batch,
                             speakers=example_speakers,
                             sample_rate=sample_rate)
print(audio_paths)

## V1

We have received a lot of questions regarding the packaging requirements and utils from the `silero-models` repo from people trying to run models locally standalone (on their desktop for example). The project is packaged using `torch.hub` utils which basically are in the `hubconf.py` [file](https://github.com/snakers4/silero-models/blob/master/hubconf.py) and `tts_utils.py` [file](https://github.com/snakers4/silero-models/blob/master/src/silero/tts_utils.py).

For some reason this is very difficult to understand for some users. Also the `hubconf.py` contains some dependecies, which may not be necessarily required when running TTS for example on your desktop, i.e. `torchaudio` and `omegaconf`.

The following example is a minimal standalone example for such a use-case. It has very little external dependecies (essentially just `torch`, the rest is just standard python library). It basically does the following:

- Loads one of the 16 kHz models (I just chose one randomly), listed on the models.yml [file](https://github.com/snakers4/silero-models/blob/master/models.yml) locally and uses it as cache;
- The `symbols` are taken from the same  models.yml file;
- `apply_tts` is just one of the utils we provide in the project;
- The rest is self-explanatory;

In order to use this example, you will need to handle the resulting audios by yourself.

In [ ]:
import os
import wave
import torch
import contextlib


torch.set_grad_enabled(False)
device = torch.device('cpu')
torch.set_num_threads(4)  # safe optimal value, i.e. 2 CPU cores, does not work properly in colab
symbols = '_~абвгдеёжзийклмнопрстуфхцчшщъыьэюя +.,!?…:;–'
local_file = 'model.jit'


if not os.path.isfile(local_file):
  torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v1_kseniya_16000.jit',
                                 local_file)

if not os.path.isfile('tts_utils.py'):
  torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/src/silero/tts_utils.py',
                                 'tts_utils.py')
  from tts_utils import apply_tts  # modify these utils and use them your project
  

model = torch.jit.load('model.jit',
                       map_location=device)
model.eval()
example_batch = ['В н+едрах т+ундры в+ыдры в г+етрах т+ырят в в+ёдра +ядра к+едров.',
                 'К+отики - +это ж+идкость!',
                 'М+ама М+илу м+ыла с м+ылом.']
sample_rate = 16000
model = model.to(device)

audio = apply_tts(texts=example_batch,
                  model=model,
                  sample_rate=sample_rate,
                  symbols=symbols,
                  device=device)

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

for i, _audio in enumerate(audio):
  write_wave(path=f'test_{str(i).zfill(3)}.wav',
             audio=(audio[i] * 32767).numpy().astype('int16'),
             sample_rate=16000)

In [ ]:
from IPython.display import Audio, display
display(Audio('test_000.wav', rate=16000))
display(Audio('test_001.wav', rate=16000))
display(Audio('test_002.wav', rate=16000))